In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 18 20:18:28 2019

@author: ayan
"""

import numpy as np
import pandas as pd
import math


def entropy(S):
    p1=list(S[:,3]).count('yes')/(S.shape[0])
    p0=list(S[:,3]).count('no')/(S.shape[0])
    if(p0==0): return -p1*math.log2(p1)
    if(p1==0): return -p0*math.log2(p0)
    return -p0*math.log2(p0)-p1*math.log2(p1)

def gain(S,A):
    """
       S-> data
       A-> splitting attribute
    """
    unique_values=np.unique(S[:,A])
    S_len=S.shape[0]
    entropy_Sv=0
    for v in unique_values:
        Sv=S[S[:,A]==v]
        entropy_Sv+=(Sv.shape[0]/S_len)*entropy(Sv)
    
    return entropy(S)-entropy_Sv

class node:
    
    def __init__(self,data,value=None,splitting_attr=None,branches=[],leaf=False,predict=None):
        self.value=value
        self.splitting_attr=splitting_attr
        self.branches=branches
        self.leaf=leaf
        if(leaf):
            self.predict=predict
class Tree:
    def __init__(self,root=None):
        self.root=root
        
    def build_tree(self,training,features,max_tree_depth=4,value=None,l=0):
    
        p1=list(training[:,3]).count('yes')
        p0=list(training[:,3]).count('no')

        if(p1==0): return node(training,value,leaf=True,predict='no')
        if(p0==0): return node(training,value,leaf=True,predict='yes')
        if(features.shape[0]==0 ):
            if(p0>p1): return node(training,value,leaf=True,predict='no')
            else: return node(training,value,leaf=True,predict='yes')
       
        max_gain=-1
        splitting_attr=''
        for i in features:
            ig=gain(training,attribute[i])
            if(ig>max_gain):
                max_gain=ig
                splitting_attr=i
        branches=[]
        for v in np.unique(training[:,attribute[splitting_attr]]):
            if(training[training[:,attribute[splitting_attr]]==v].size==0): 
                    if(counts[0]>counts[1]): branches.append(node(training,value=v,leaf=True,predict='no'))
                    else: branches.append( node(training,value=v,leaf=True,predict='yes'))
            else:
                 branches.append(self.build_tree(training[training[:,attribute[splitting_attr]]==v],features[features!=splitting_attr],
                                      max_tree_depth,value=v,l=l+1) )
            
        root=node(training,value,splitting_attr,branches)
        self.root=root
        return root
  
    def Prediction(self,test):
        r=self.root
        while(r.leaf!=True):
            a=r.splitting_attr
            for node in r.branches:
                if(test[attribute[a]]==node.value):
                    r=node
                    break
        return r.predict
    def print_tree(self,root,ind='  ',l=0,a=None):
        if(root.leaf==True):
            if(a!=None):
                print(l*ind,a," = ",root.value,": ",root.predict)
                return
            else: 
                print(l*ind,"survived = ",root.predict)
                return
        if(a!=None): print(l*ind,a," = ",root.value)
        a=root.splitting_attr
        for node in root.branches:
            self.print_tree(node,'  ',l+1,a)

data=pd.read_csv("data1_19.csv")
X=data.to_numpy()
indices = np.random.permutation(X.shape[0])
test_idx, training_idx = indices[:300], indices[300:]
training, test = X[training_idx,:], X[test_idx,:]
features=np.array(['pclass','age','gender'])
attribute={'pclass':0,'age':1,'gender':2,'survived':3}
        
tree=Tree()
root=tree.build_tree(training,features)
tree.print_tree(root)

   gender  =  female
     pclass  =  1st
       age  =  adult :  yes
       age  =  child :  yes
     pclass  =  2nd
       age  =  adult :  yes
       age  =  child :  yes
     pclass  =  3rd
       age  =  adult :  no
       age  =  child :  yes
     pclass  =  crew
       age  =  adult :  yes
   gender  =  male
     pclass  =  1st
       age  =  adult :  no
       age  =  child :  yes
     pclass  =  2nd
       age  =  adult :  no
       age  =  child :  yes
     pclass  =  3rd
       age  =  adult :  no
       age  =  child :  no
     pclass  =  crew
       age  =  adult :  no


In [2]:
s=0
for i in range(len(test)):
    if(test[i,3]==tree.Prediction(test[i])):
        s+=1
print(s," ",s/300)

237   0.79
